# Bayesian grey-box identification with Gaussian Process State-Space Model

Last update: 20-10-2023

---

The primary goal of this project is to identify parameters in a grey-box model of a thermal setup (see system description below), based on temperature measurements at specific locations on the system. A secondary goal is to develop expertise in Bayesian inference techniques for system identification in general.

In [ ]:
import Pkg
Pkg.activate("..")
Pkg.instantiate()

## System description

A schematic depiction of the setup is shown in the figure below. 

<p><center><img src='../../system/system-photo.png'/></center></p>

In short, the setup consists of 3 metal blocks which have been lined up, with resistive nylon pads interposed. The temperature can be measured using thermistors at arbitrary places on the setup; for simplicity we assume that we measure the temperature at a single spot on each block, which we call $\tau_1$, $\tau_2$, and $\tau_3$. The temperatures will evolve due to a number of different factors; we will only consider the influence of conduction, convection, radiation, and the user controlled input heat (band heaters).

By assuming that conduction within blocks is so fast that there are no temperature differences within a block, we may model the system using a [lumped-element model](https://en.wikipedia.org/wiki/Lumped-element_model), governed by the following system of ODEs:

$$\frac{d}{dt}\begin{pmatrix} m_1 c_{p, 1} \tau_1 \\ m_2 c_{p, 2} \tau_2 \\ m_3 c_{p, 3} \tau_3 \end{pmatrix} = 
\underbrace{\begin{pmatrix} -k_{12} & k_{12} & 0 \\ k_{12} & -(k_{12} + k_{23}) & k_{23} \\ 0 & k_{23} & -k_{23} \end{pmatrix} \begin{pmatrix} \tau_1 \\ \tau_2 \\ \tau_3 \end{pmatrix}}_{\textrm{conduction}} + \underbrace{\begin{pmatrix} h(\tau_1, \tau_a, 1, t) \\ h(\tau_2, \tau_a, 2, t) \\ h(\tau_3, \tau_a, 3, t) \end{pmatrix}}_{\textrm{convection}} + \underbrace{\sigma \begin{pmatrix} a_1 \varepsilon_1 (\tau_a^4 - \tau_1^4) \\ a_2 \varepsilon_2 (\tau_a^4 - \tau_2^4) \\ a_3 \varepsilon_3 (\tau_a^4 - \tau_3^4) \end{pmatrix}}_{\textrm{radiation}} + \underbrace{\begin{pmatrix} u_1 \\ u_2 \\ u_3 \end{pmatrix}}_{\textrm{input}}.$$

Convection is notoriously hard to model. A coarse approximation would be Newton's law of cooling (Clercx, 2015; Eq. 8.17), which states that convection is linear in the difference between the temperature of the block and the ambient temperature: $h_a (\tau_a - \tau_i)$. With this linear term, we can take steps similar to the identification of the oscillator in Rogers \& Friis (2022), describing the nonlinear function as the combination of a linear term and a "nonlinear remainder":

$$\underbrace{h(\tau_i, \tau_a, i, t)}_{\text{total convection}} = \underbrace{h_a (\tau_a - \tau_i)}_{\text{linear cooling law}} + \underbrace{r(\tau_i, \tau_a, i, t)}_{\text{nonlinear remainder}} \, ,$$

for some constant $h_a$. Furthermore, the role of radiation can often be neglected.

We can absorb the ambient temperature and the linear convection term into the state vector. Now, the governing equations become:

$$\begin{aligned}
\begin{bmatrix} \dot{\tau}_a \\ \dot{\tau}_1 \\ \vdots \\ \dot{\tau}_3 \end{bmatrix} = \begin{bmatrix} 0 & 0 & 0 & 0 \\ \frac{h_a a_1}{m_1 c_{p,1}} & \frac{-k_{12} - h_a a_1}{m_1 c_{p,1}} & \frac{k_{12}}{m_1 c_{p,1}} & 0 \\
\frac{h_a a_2}{m_2 c_{p,2}} & \frac{k_{12}}{m_2 c_{p,2}} & \frac{-k_{12} - k_{23} - h_a a_2}{m_2 c_{p,2}} & \frac{k_{23}}{m_2 c_{p,2}} \\
\frac{h_a a_3}{m_3 c_{p,3}} & 0 & \frac{k_{23}}{m_3 c_{p,3}} & \frac{-k_{23} - h_a a_3}{m_3 c_{p,3}} \end{bmatrix} \begin{bmatrix} \tau_a \\ \tau_1 \\ \tau_2 \\ \tau_3 \end{bmatrix} + \begin{bmatrix} r(\tau_1, \tau_a, 1, t) \\ r(\tau_2, \tau_a, 2, t) \\ r(\tau_3, \tau_a, 3, t) \end{bmatrix} + \begin{bmatrix} 0 & 0 & 0 \\ \frac{1}{m_1 c_{p,1}} & 0 & 0 \\ 0 & \frac{1}{m_2 c_{p,2}} & 0 \\ 0 & 0 & \frac{1}{m_3 c_{p,3}} \end{bmatrix} \begin{bmatrix} u_1 \\ u_2 \\ u_3 \end{bmatrix}  \, .
\end{aligned}$$

In these equations, we can distinguish three types of quantities:
1. Measured/observed quantities: e.g. $\tau_a$, $\tau_i$, $u_i$. These may vary over tsteps, and are known up to a given accuracy due to measurement noise;

2. Known constants: e.g. $m_i$, $c_{p, i}$, $a$. These are fully known, and are constant over tsteps. This is reasonable for quantities such as mass $m$ and surface area $a$ (which can be easily measured) and specific heat capacity $c_p$;

3. Unknown constants: e.g. $k_{ij}$, $h_a$. Yhere is no simple physical way to measure or derive their values. For example, the conduction coefficients $k_{ij}$ can vary depending on how tightly the blocks have been put together. In this project, we want to identify these constants using Bayesian inference.

In [ ]:
using Revise
using RxInfer
using Random
using MAT
using Optim
using JLD2
using LaTeXStrings
using Polynomials
using DifferentialEquations
using Distributions
using LinearAlgebra
using Plots; default(label="", linewidth=3, margin=15Plots.pt)

include("../util/conductance.jl")
include("../util/discretization.jl")

In [ ]:
## Load data
dat = matread("data/20230110_143005_sensortraces.mat")

In [ ]:
# Sensors per block
Ns = convert.(Int64, dat["N"])
N = sum(Ns)

# Sensor 7 is broken
NTCIX = [1:6; 8:13];
measurements = hcat([dat["NTC$k"] for k in NTCIX]...)
inputs = dat["u"]; # First column is ambient

In [ ]:
# Time parameters
Ts = dat["Ts"]
tmax = convert(Int64, dat["tmax"])
tmax = 28_800

# Subsample
ssix = 1
Δt = Ts*ssix
tsteps = range(0.0, step=Δt, stop=tmax-1)
T = length(tsteps);
inputs_ss = inputs[1:ssix:tmax,:]
measurements_ss = measurements[1:ssix:tmax,:];

In [ ]:
lcolors = hcat([repeat(["red"], Ns[1]); repeat(["blue"], Ns[2]); repeat(["orange"], Ns[3])]...)
labelsm = hcat([L"$T_{%$k}$" for k in NTCIX]...)

vis_ss = 1_000

plt1 = plot(tsteps[2:vis_ss:end] ./ 3600, 
    inputs_ss[2:vis_ss:end,1],
    label=L"$T_a$",
    color="black",
    )             
plot!(tsteps[2:vis_ss:end] ./ 3600, 
    measurements_ss[2:vis_ss:end,:],
    linecolors = lcolors, 
    linewidth=4,
    labels = labelsm,
    # xscale=:log10,
    ylabel="temperature [C]",
    legend=:topleft,
    xticks=([1., 2. , 4., 8.]),
    xlabel="time [h]",
    ylims=[20., 50.],
    grid=true,
    size=(900,500),
    legendfontsize=10,
    guidefontsize=10,)
        

In [ ]:
vis_ss = 1_000
vis_ix = [3,9,12]

lcolors3 = ["red" "blue" "orange"]
labelsm3 = hcat([L"$T_{%$k}$" for k in vis_ix]...)

plt1 = plot(tsteps[2:vis_ss:end] ./ 3600, 
            measurements_ss[2:vis_ss:end,vis_ix],
            linecolors = lcolors3, 
            linewidth=4,
            labels = labelsm3,
            # xscale=:log10,
            ylabel="temperature [C]",
            legend=:bottom,
            # xticks=([log10(8.), log10(16.), log10(24.)], [8., 16., 24.]),
            ylims=[20., 50.],
            grid=true)
plt2 = plot(tsteps[2:vis_ss:end] ./ 3600, 
            inputs_ss[2:vis_ss:end,1],
            xlabel="time [h]",
            color="black",
            yticks=[23, 24],
            ylims=[23, 24],
            label=L"$T_a$",
            ylabel="temperature [C]",
            legend=:top)            
plot(plt1,plt2, layout=grid(2,1, heights=[0.6, 0.4]), margin=0.1Plots.mm, size=(500,350))

In [ ]:
mcps = dat["Mcps"]         # Mass heat capacity
gc   = dat["gc"]           # Base conduction of blocks
Ac   = dat["Ac"]           # Cross-section surfaces (m2)
An   = dat["An"]           # Outer surface areas (m2)
l    = dat["l"]            # Lengths
lpl  = dat["l_pom_left"]
lpr  = dat["l_pom_right"]

An_ = vcat([repeat([An[k]], Ns[k]) for k in 1:3]...)
M   = diagm(vcat([repeat([mcps[k]], Ns[k]) for k in 1:3]...))

In [ ]:
k_12, k_23, h_a = dat["xoptimal"]

## GP-augmented state-space model

### Matern-1/2 GP in state-space form
Suppose a function $h(x)$ follows a zero-mean Gaussian Process

$$\begin{aligned}
h(x) \sim \mathcal{GP}(0, k(x,x')) \, .
\end{aligned}$$

The GP can be written as the following differential equation ([Hartikainen, 2013)](https://aaltodoc.aalto.fi/bitstream/handle/123456789/7579/isbn9789526049847.pdf?sequence=1&isAllowed=y)):

$$\begin{align}
\dot{h} = F_h h + L v \, ,
\end{align}$$

with $h$ being states, $F_h$ a state transition matrix and $v$ a white noise process. The Matern GP has a stationary covariance function, defined as:

$$\begin{aligned}
k(\tau) = \sigma^2 \frac{2^{1-\nu}}{\Gamma(\nu)} \left(\frac{\sqrt{2\nu}\tau}{l} \right)^\nu K_\nu\left(\frac{\sqrt{2\nu}\tau}{l} \right)
\end{aligned}$$

where $\sigma^2$ is a scale hyperparameter, $l$ a characteristic length-scale, $\nu$ the smoothness hyperparameter, and $K_\nu(.)$ is a modified Bessel function of the second kind. 

Matern-1/2 refers to $\nu=1/2$. The matrices for the state space model are computed as follows:

$$\begin{aligned}
\mathbf{F}_h = -\lambda,\quad \quad \mathbf{L} = 1, \quad \quad \mathbf{P}_\infty = \sigma^2  ,\quad \quad  Q_c = 2\lambda\sigma^2
\end{aligned}$$ 

where $\lambda = \frac{\sqrt{3}}{l} $.

### Augmented state-space model

The original state-space model (see top) is:

$$\begin{aligned}
\underbrace{\begin{bmatrix} \dot{\tau}_1 \\ \vdots \\ \dot{\tau}_3 \end{bmatrix}}_{\dot{z}} = \underbrace{\begin{bmatrix} \frac{-k_{12} - h_a a_1}{m_1 c_{p,1}} & \frac{k_{12}}{m_1 c_{p,1}} & 0 \\
\frac{k_{12}}{m_2 c_{p,2}} & \frac{-k_{12} - k_{23} - h_a a_2}{m_2 c_{p,2}} & \frac{k_{23}}{m_2 c_{p,2}} \\
 0 & \frac{k_{23}}{m_3 c_{p,3}} & \frac{-k_{23} - h_a a_3}{m_3 c_{p,3}} \end{bmatrix}}_{F} \underbrace{\begin{bmatrix} \tau_1 \\ \vdots \\ \tau_3 \end{bmatrix}}_{z} + \underbrace{\begin{bmatrix} \frac{h_a a_1}{m_1 c_{p,1}} & \frac{1}{m_1 c_{p,1}} & 0 & 0 \\ \frac{h_a a_2}{m_2 c_{p,2}} & 0 & \frac{1}{m_2 c_{p,2}} & 0 \\ \frac{h_a a_3}{m_3 c_{p,3}} & 0 & 0 & \frac{1}{m_3 c_{p,3}} \end{bmatrix}}_{G} \underbrace{\begin{bmatrix} \tau_a \\ u_1 \\ u_2 \\ u_3 \end{bmatrix}}_{u} + M^{-1}r(z) \, 
\end{aligned}$$

Since $r(z)$ is vector-valued, we need multiple GP functions:

$$\begin{aligned}
r(z) \approx h = \begin{bmatrix} h^{(1)} \\ h^{(2)} \\ h^{(3)} \end{bmatrix} \, .
\end{aligned}$$

Augmenting the state-space model with $h$ as described above, gives:

$$\begin{aligned}
\begin{bmatrix} \dot{z} \\ \dot{h} \end{bmatrix} = \begin{bmatrix} F & M^{-1} \\ 0 & F_h \end{bmatrix} \begin{bmatrix} z \\ h \end{bmatrix} +  \begin{bmatrix} G \\ 0 \end{bmatrix} u + \begin{bmatrix} 0 \\ L \end{bmatrix} w \, .
\end{aligned}$$

After discretization, we get:

$$\begin{aligned}
x_{k+1} = A x_k + B u_k + w_k\, , \quad \text{with}\ w_k \sim \mathcal{N}(0, Q) \, ,
\end{aligned}$$

where $x_k = \begin{bmatrix} z_k \\ h_k \end{bmatrix}$ and

$$\begin{aligned}
    A = \exp\big(\Delta t \begin{bmatrix} F & M^{-1} \\ 0 & F_h \end{bmatrix} \big) \, , \quad B = \begin{bmatrix} \Delta t G \\ 0 \end{bmatrix} \, , \quad Q = \int_0^{\Delta t} \exp(\begin{bmatrix} F & M^{-1} \\ 0 & F_h \end{bmatrix}t) \begin{bmatrix} 0 \\ L \end{bmatrix} Q_c \begin{bmatrix} 0 \\ L \end{bmatrix}^{\top} \exp(\begin{bmatrix} F & M^{-1} \\ 0 & F_h \end{bmatrix} t)^{\top} dt \, .
\end{aligned}$$


In [ ]:
@model function SSM(A, B, C, Q, R, m0, S0; T=1)
    
    x = randomvar(T)
    u = datavar(Vector{Float64}, T)
    y = datavar(Vector{Float64}, T)
    
    x_0 ~ MvNormalMeanCovariance(m0, S0)
    x_kmin1 = x_0
    for k = 1:T
        
        x[k] ~ MvNormalMeanCovariance(A*x_kmin1 + B*u[k], Q)
        y[k] ~ MvNormalMeanCovariance(C*x[k], R)
        
        x_kmin1 = x[k]
    end
end

In [ ]:
Dx = 24
Dt = 12
Dh = 12
Du = 13
Dy = 12

F  = inv(M)*conductances([k_12,k_12,h_a], Ns, An, gc, [lpl, lpr])
B  = [Δt*inv(M)*[h_a*An_ diagm(ones(N))]; zeros(Dh,Du)]
C  = [diagm(ones(Dy)) zeros(Dy,Dx-Dy)]
R  = diagm(1e-3*ones(Dy))
m0 = [measurements_ss[1,:]; zeros(Dh)];

In [ ]:
output_ = [measurements_ss[k,:] for k in 1:T];
inputs_ = [inputs_ss[k,:] for k in 1:T];

In [ ]:
function J(hparams::AbstractVector)
    "hparams = [λ, σ]"
    
    A = [F                                  inv(M);
         zeros(Dh,Dh)  diagm(-hparams[1]*ones(Dh))]*Δt + diagm(ones(Dx))
    
    Q  = analyticQ(inv(M), hparams[1], hparams[2], Δt=Δt)
    S0 = diagm([1e-8*ones(Dt); hparams[2]^2*ones(Dh)])

    results = infer(
        model       = SSM(A, B, C, Q, R, m0,S0, T=T),
        data        = (y = output_, u = inputs_),
        options     = (limit_stack_depth = 100,),
        free_energy = true
    )
    return results.free_energy[end]    
end
        
ops = Optim.Options(g_tol=1e-8, time_limit=300, show_every=1)
res = optimize(J, 1e-8, 1e8, [1e4, 1e3], Fminbox(LBFGS()), ops; autodiff=:forward)
λ_star,γ_star = Optim.minimizer(res)

In [ ]:
# λ_star  = 2.8902590487600577e-7
# σ²_star = 1002.7080959640236

λ_star =  17.190528806217117
γ_star = 0.00015355491941274524

In [ ]:
# State estimation with optimized hyperparameters
A = exp([F                              inv(M);
         zeros(Dh,Dh)  diagm(-λ_star*ones(Dh))]*Δt)

Q = analyticQ(inv(M), λ_star, γ_star, Δt=Δt)
S0 = diagm([1e-3*ones(Dy); γ_star^2*ones(Dh)])

results = infer(
    model        = SSM(A, B, C, Q, R, m0, S0, T=T),
    data         = (y = output_, u = inputs_),
    options      = (limit_stack_depth = 100,),
    free_energy  = true,
    showprogress = true,
)

In [ ]:
qx = results.posteriors[:x]
fitx_v = hcat( var.(qx)...)
fitx_m = hcat(mean.(qx)...)

In [ ]:
labelsf = hcat(["τ_$k fit" for k in NTCIX]...)

vis_ss = 1_000

plot(tsteps[2:vis_ss:end],
     fitx_m[1:12,2:vis_ss:end]';
     ribbon=sqrt.(fitx_v[1:12,2:vis_ss:end])',
     legend = :topleft, 
     linecolors = lcolors, 
     labels = labelsf,
     xlabel = "tsteps [s]", 
     ylabel = "temperature [C]",
     size=(900,400)
)
plot!(tsteps[2:vis_ss:end], 
      measurements_ss[2:vis_ss:end,:], 
      alpha = 0.3,
      linestyle = :dash,
      linecolors = lcolors, 
      labels = labelsm,
#       xscale=:log10,
)

### Gaussian process fits over tsteps

In [ ]:
ntc_ix = 3
vis_ss = 100

scatter3d(inputs_ss[2:vis_ss:end,1],
         fitx_m[ntc_ix,2:vis_ss:end],
         fitx_m[12+ntc_ix,2:vis_ss:end],
         legend = true, 
        color = "red",
#      labels = "GP NTC#$ntc_ix",
        title ="NTC#$ntc_ix",
        ylabel = "Block temperature [C]",
        xlabel = "Ambient temperature [C]",
        zlabel = "Change in temperature [ΔC]",
      #  camera = (90,20),
        size=(600,500)
)

In [ ]:
ntc_ix = 1
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "red",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$ntc_ix",
     xlabel = "tsteps [s]",
     # xscale = :log10,
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 2
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "red",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$ntc_ix",
     xlabel = "tsteps [s]",
     # xscale = :log10,
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 3
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "red",
#      labels = "GP NTC#$ntc_ix",
     # xscale=:log10,
     title ="NTC#$ntc_ix",
     xlabel = "tsteps [s]",
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 4
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "red",
#      labels = "GP NTC#$ntc_ix",
      title ="NTC#$ntc_ix",
     xlabel = "tsteps [s]",
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 5
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "red",
#      labels = "GP NTC#$ntc_ix",
     xscale=:log10,
     title ="NTC#$ntc_ix",
     xlabel = "tsteps [s]",
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 6
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "blue",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$ntc_ix",
     xlabel = "tsteps [s]", 
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 7
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "blue",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$(ntc_ix+1)",
     xlabel = "tsteps [s]", 
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 8
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "blue",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$(ntc_ix+1)",
     xlabel = "tsteps [s]", 
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 9
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "blue",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$(ntc_ix+1)",
     xlabel = "tsteps [s]", 
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 10
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "orange",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$(ntc_ix+1)",
     xlabel = "tsteps [s]", 
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 11
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "orange",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$(ntc_ix+1)",
     xlabel = "tsteps [s]", 
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

In [ ]:
ntc_ix = 12
plot(tsteps[2:vis_ss:end],
     fitx_m[12+ntc_ix,2:vis_ss:end]*Δt./M[ntc_ix,ntc_ix];
     # ribbon=sqrt.(fitx_v[12+ntc_ix,2:vis_ss:end])*Δt./M[ntc_ix,ntc_ix],
     legend = true, 
     color = "orange",
#      labels = "GP NTC#$ntc_ix",
     title ="NTC#$(ntc_ix+1)",
     xlabel = "tsteps [s]", 
     ylabel = "Change in temperature [ΔC]",
     size=(600,400)
)

### Extract nonlinearity

In [ ]:
deg = 3
h_pol = []

# Set higher weights for transients
polweights = [10*ones(1_000); ones(4_000); 1e-1*ones(5_000); 1e-2*ones(T-10_000)]

for nn in 1:N
    push!(h_pol, Polynomials.fit(fitx_m[nn,:], fitx_m[N+nn,:], deg; weights=polweights))
end

In [ ]:
ntc_ix = 1

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$ntc_ix"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.1, color="red")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="black", linestyle=:dash, label="polynomial fit")

In [ ]:
ntc_ix = 2

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$ntc_ix"
)
plot!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:]*Δt./M[ntc_ix,ntc_ix], ribbon=0.0*sqrt.(fitx_v[N+ntc_ix,:]*Δt./M[ntc_ix,ntc_ix]), fillalpha=0.3, color="darkred")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x)*Δt./M[ntc_ix,ntc_ix], color="red", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 3

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$ntc_ix"
)
plot!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], ribbon=0.0sqrt.(fitx_v[N+ntc_ix,:]*Δt./M[ntc_ix,ntc_ix]), fillalpha=0.3, color="darkred")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="red", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 4

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$ntc_ix"
)
plot!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:]*Δt./M[ntc_ix,ntc_ix], ribbon=0.0sqrt.(fitx_v[N+ntc_ix,:]*Δt./M[ntc_ix,ntc_ix]), fillalpha=0.3, color="darkred")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x)*Δt./M[ntc_ix,ntc_ix], color="red", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 5

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$ntc_ix"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.3, color="red")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="red", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 6

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$ntc_ix"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.3, color="blue")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="blue", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 7

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$(ntc_ix+1)"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.3, color="blue")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="blue", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 8

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$(ntc_ix+1)"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.3, color="blue")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="blue", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 9

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$(ntc_ix+1)"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.3, color="blue")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="blue", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 10

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$(ntc_ix+1)"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.3, color="orange")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="orange", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 11

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$(ntc_ix+1)"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.3, color="orange")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="orange", linestyle=:dash, label="polynomial")

In [ ]:
ntc_ix = 12

plot(xlabel="temperature [C]",
     ylabel="change in temperature [ΔC]",
     title ="NTC#$(ntc_ix+1)"
)
scatter!(fitx_m[ntc_ix,:], fitx_m[N+ntc_ix,:], markersize=100fitx_v[N+ntc_ix,:], alpha=0.3, color="orange")
plot!(sort(fitx_m[ntc_ix,:]), x -> h_pol[ntc_ix](x), color="orange", linestyle=:dash, label="polynomial")

## Simulation

In [ ]:
sim_temps  = zeros(T,Dt)
id_nlconv = zeros(T,Dt)

sim_temps[1,:] = dat["T0"]
for ii in 2:T

    id_nlconv[ii,:] = [h_pol[k](sim_temps[ii-1,k]) for k in 1:12]

    sim_temps[ii,:] = A[1:Dt,1:Dt]*sim_temps[ii-1,:] + B[1:Dt,:]*inputs_ss[ii,:] + Δt*inv(M)*id_nlconv[ii,:]
end

In [ ]:
SIM_MSE = mean( (sim_temps .- measurements_ss).^2 )

In [ ]:
labelss = hcat(["τ_$k simulated" for k in NTCIX]...)

plot(tsteps[2:vis_ss:end],
     sim_temps[2:vis_ss:end,:],
     legend = true, 
     linecolors = lcolors, 
     labels = labelss,
     xlabel = "tsteps [s]", 
     ylabel = "temperature [C]",
     size=(900,400),
     title="MSE = $SIM_MSE",
     # xscale=:log10,
)
# plot!(tsteps[2:vis_ss:end], 
#       measurements_ss[2:vis_ss:end,:], 
#       alpha = 0.5,
#       linecolors = lcolors, 
#       linestyle = :dash,
# )

In [ ]:
plot(tsteps[2:vis_ss:end] ./ 3600,
     sim_temps[2:vis_ss:end,:] .- measurements_ss[2:vis_ss:end,:],
     legend = true, 
     linecolors = lcolors, 
     # labels = labelss,
     xlabel = "time [h]", 
     ylabel = "simulations - measurements",
     size=(900,400),
     title="MSE = $SIM_MSE",
     # xscale=:log10,
)

In [ ]:
jldsave("results/GPASSM.jld2"; M,F,A,B,C,Q,R, measurements_ss, inputs_ss, tsteps, N, Δt, fitx_m, fitx_v, h_pol, λ_star, γ_star, sim_temps)

In [ ]:
# matwrite("results/GP-SSM.mat", 
#     Dict("A" => A[1:Dt,1:Dt],
#          "B" => B[1:Dt,:],
#          "Q" => Q[1:Dt,1:Dt],
#          "h1_pol"  => h_pol[1].coeffs,
#          "h2_pol"  => h_pol[2].coeffs,
#          "h3_pol"  => h_pol[3].coeffs,
#          "h4_pol"  => h_pol[4].coeffs,
#          "h5_pol"  => h_pol[5].coeffs,
#          "h6_pol"  => h_pol[6].coeffs,
#          "h8_pol"  => h_pol[7].coeffs,
#          "h9_pol"  => h_pol[8].coeffs,
#          "h10_pol" => h_pol[9].coeffs,
#          "h11_pol" => h_pol[10].coeffs,
#          "h12_pol" => h_pol[11].coeffs,
#          "h13_pol" => h_pol[12].coeffs,
#          "pol_degree" => deg,
#          "y_sim" => sim_temp,
#          "Ts" => Δt)
# )

In [ ]:
# matwrite("results/GP-SSM-inference.mat", 
#     Dict("A" => A,
#          "B" => B,
#          "C" => C,
#          "Q" => Q,
#          "states_mean" => fitx_m,
#          "states_vars" => fitx_v,
#          "h1_pol"  => h_pol[1].coeffs,
#          "h2_pol"  => h_pol[2].coeffs,
#          "h3_pol"  => h_pol[3].coeffs,
#          "h4_pol"  => h_pol[4].coeffs,
#          "h5_pol"  => h_pol[5].coeffs,
#          "h6_pol"  => h_pol[6].coeffs,
#          "h8_pol"  => h_pol[7].coeffs,
#          "h9_pol"  => h_pol[8].coeffs,
#          "h10_pol" => h_pol[9].coeffs,
#          "h11_pol" => h_pol[10].coeffs,
#          "h12_pol" => h_pol[11].coeffs,
#          "h13_pol" => h_pol[12].coeffs,
#          "pol_degree" => deg,
#          "y_sim" => sim_temp,
#          "Ts" => Δt)
# )